# Functionality: Visuliation of graph

### function rescale_minmax: rescale data

$$
    data_{minsubs} = data - min(data)
$$


$$
    data_{rescaled} = (\frac{data_{minsubs}}{max(data_{minsubs}) - min(data_{minsubs})} + min_{scale}) * int_{len} \\
    default: min_{scale} = 0, int_{len} = 1
$$

<font color='red'> Not quite clear why using $min_{scale}$ and $int_{len}$ </font>

In [ ]:
def rescale_minmax(data, min_scale=0, int_len=1):
    """
        rescale data 
    """
    
    data -= np.min(np.array(data).flatten())
    data /= (np.max(data.flatten()) - np.min(data.flatten()))

    data += min_scale
    data *= int_len
    return data

### function checkSymmetric

In [ ]:
def checkSymmetric(w, diagfill=0, tol=1e-8):
    def checkSymmetric2D(w_sin, diagfill=0, tol=1e-8):
        flag = np.allclose(w_sin, w_sin.transpose(), atol=tol)
        if not flag:
            w_sin *= np.tri(*w_sin.shape)
            w_sin = w_sin + w_sin.T
            np.fill_diagonal(w_sin, diagfill)
        return flag, w_sin

    flag = True
    if len(w.shape) > 2:
        for i in range(w.shape[2]):
            flag_sin, w[:, :, i] = checkSymmetric2D(w[:, :, i], diagfill=diagfill, tol=tol)
            if not flag_sin:
                flag = flag_sin
    else:
        flag, w = checkSymmetric2D(w, diagfill=diagfill, tol=tol)

    return flag, w

### function create_graph

In [ ]:
def create_graph(weights, labels=None, selfloop=False, directed = False, rescale=True, lowbnd=0, wmin=0, wlen=1):
    """
        
        
        Args:
            weights: the edge weight
            
            labels
            
            selfloop:
            
            tagdirected: if directed (True) or not (False)
            
            rescale:
            
            lowbnd: lower boundary to show (i.e only show the edges with weight larger than lowbnd)
        
    """
    
    # set 0 if weights < lowbnd
    w = np.multiply(weights, weights > lowbnd)
    
    # rescale to minmax 
    if rescale:
        w = rescale_minmax(w, min_scale=wmin, int_len=wlen)
    
    flag, w = checkSymmetric(w, diagfill=selfloop)
    
    # generate a graph from its adjacency matrix
    g = ig.Graph.Adjacency((w>0).tolist())
    
    
    if not directed:
        # return an undirected copy of this graph
        g = g.as_undirected()
        
        wght = w * np.tri(* w.shape)
    else:
        wght = w
    
    g.es['weight'] = wght[wght.nonzero()]
    g.es['width'] = wght[wght.nonzero()]
    
    if labels is not None:
        g.vs['name'] = labels
        g.vs['label'] = labels

    return g, w

In [ ]:
def graph_style(g, layout, labels=None):
    """
        define the visual style of the graph
        
        Args:
            g
        
    """
    # define the visual style
    visual_style = dict()
    visual_style['vertex_color'] = 'black'
    visual_style['vertex_label_size'] = 10
    visual_style['vertex_label_dist'] = 2
    visual_style['vertex_label_color'] = 'black'
    
    try:
        visual_style['vertex_label'] = g.vs['label']
    except KeyError:
        if labels is not None:
            if type(labels) is np.ndarray:
                labels = labels.tolist()
            
            visual_style['vertex_label'] = labels
            
    visual_style['edge_width'] = g.es['weight']
    visual_style['layout'] = layout
    outdegree = g.outdegree()
    visual_style["vertex_size"] = [x/max(outdegree)*10+5 for x in outdegree]
    
    return visual_style    

In [ ]:
def graph_plot(g, layout, clustering=None, savename = 'network.png'):
    visual_style = graph_style(g,layout)
    if clustering is not None:
        cluster_idx = find_comty(g,clustering)
        palette = palettes.ClusterColoringPalette(len(np.unique(cluster_idx)))
        g.vs['color'] = [palette[cluster_idx[x]] for x in range(len(cluster_idx))]
        visual_style['vertex_color'] = g.vs['color']
    else:
        visual_style['vertex_color'] = 'black' 
    
    visual_style['bbox'] = [500, 500]     
    visual_style['margin'] = np.array([10,10,10,30]).tolist()  
    return ig.plot(g,savename ,**visual_style)